# G fileの読込みと整合性の確認

In [1]:
import sub.g_file as sgf
import sub.plot as spl
import plotly.express as px
import pandas as pd
import numpy as np

## G fileの読込みとフラックスの表示

In [2]:
#pr = sgf.read_g_file('g104362.00213')
pr = sgf.read_g_file('g_quest.txt')

In [3]:
spl.contour(pr['psizr'], pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

## 圧力に関する整合性の確認

In [4]:
# 正規化フラックスをフラックスに変換
def get_flux(normflux):
    return pr['simag']+(pr['sibry']-pr['simag'])*normflux
# 微分波形を計算_要素数は一つ減る
def get_diff(x, y):
    dy = np.append(y, 0)-np.append(0, y)
    dx = np.append(x, 0)-np.append(0, x)
    v = dy/dx
    return x[1:], v[1:-1]    

In [5]:
# pressureの波形をフラックスで微分して、dp/dfに一致することを確認
y = np.array(pr['pres'])
x = np.linspace(0, 1, len(y))
x2 = get_flux(x)
df = pd.DataFrame(data=np.array([x2, y]).T, columns=['flux', 'pressure'])
fig2 = px.line(df, x='flux', y='pressure')
fig2.show()
# フラックスでの微分を計算
x3, v = get_diff(x2, y)
df = pd.DataFrame(data=np.array([x3, v]).T, columns=['flux', 'calc_dp'])
fig2 = px.line(df, x='flux', y='calc_dp')
fig2.show()
y1 = pr['pprime']
df = pd.DataFrame(data=np.array([x2, y1]).T, columns=['flux', 'pprime'])
fig2 = px.line(df, x='flux', y='pprime')
fig2.show()

つまり、圧力分布からpprimeが計算できることを確認した。
pprimeでは、規格化フラックスでなく、フラックスの微分を取っていることが分かる。

## ポロイダル電流に関する整合性の確認

In [6]:
# fpolの波形をフラックスで微分して、ffprimに一致することを確認
y = np.array(pr['fpol'])
x = np.linspace(0, 1, len(y))
df = pd.DataFrame(data=np.array([x, y]).T, columns=['norm flux', 'fpol'])
fig2 = px.line(df, x='norm flux', y='fpol')
fig2.show()
x2 = get_flux(x)
# ffprimを計算
x3, v = get_diff(x2, y)
df = pd.DataFrame(data=np.array([x3, v*y[:-1]]).T, columns=['flux', 'calc_ffprim'])
fig2 = px.line(df, x='flux', y='calc_ffprim')
fig2.show()
y1 = pr['ffprim']
df = pd.DataFrame(data=np.array([x2, y1]).T, columns=['flux', 'ffprim'])
fig2 = px.line(df, x='flux', y='ffprim')
fig2.show()

fpolからffprimが計算できることが確認できた。
これも微分は、正規化フラックスでなく、フラックスから計算している。

## 全電流の確認

### ドメインの作成と確認

In [7]:
# domain matrixの作成と確認
dm = sgf.get_domain(pr, 'BBBS')
spl.heatmap(dm, pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

### 全電流を算出

全電流をpprime, ffprimから算出して、全電流に整合するかを確認する。

In [8]:
from scipy import interpolate

# pprimeの線形補間関数の作成
y = pr['pprime']
x = np.linspace(0, 1, len(y))
fnc_pprim = interpolate.interp1d(x, y)

# ffprimの線形補間関数の作成
y = pr['ffprim']
x = np.linspace(0, 1, len(y))
fnc_ffprim = interpolate.interp1d(x, y)


In [9]:
# 正規化フラックスの作成
nf = (pr['psizr']-pr['simag'])/(pr['sibry']-pr['simag'])
nf = nf.reshape(-1)

# 1.0が境界、境界があいまいな場合は1にしておく
# 0.0にしてしまうと、磁気軸の値を参照してしまう場合がある。
nf = np.array([e if 0.0 <= e <= 1.0 else 1.0 for e in nf])
nf = nf.reshape(pr['nh'], pr['nw'])
nf *= dm # ドメインの外の領域は削除

In [10]:
spl.contour(nf, pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

In [11]:
# r位置のマトリックスを作成
rm = [np.linspace(pr['rleft'], pr['rleft']+pr['rdim'], pr['nw'])]*pr['nh']
rm = np.array(rm)+10**(-7)

In [12]:
# 電流密度分布の作成 
# ポロイダルカレントをu0で割っていることに注意
jt = rm*fnc_pprim(nf)*dm + fnc_ffprim(nf)/rm*dm/(4*np.pi*1.0e-7)
jt *= pr['dr']*pr['dz']

In [13]:
spl.contour(jt, pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

In [14]:
# トータルの電流値とg_fileでの電流値
jt.sum(), pr['current']

(100012.6093089286, 100000.0)

## fpolの値の確認

fpolはトロイダルコイル電流を含めたポロイダル電流である。  
従って、最外殻磁気面上でのポロイダル電流は、トロイダルコイル電流によるポロイダル電流に一致することを確認する。

In [15]:
# トロイダル磁場の値と、その位置
pr['bcentr'], pr['rcentr']

(0.16, 1.0)

In [16]:
# b=(b0 r0)/rであってプラズマの外のb0 r0
pr['bcentr']*pr['rcentr']

0.16

In [17]:
# fpolの最外殻磁気面の位置で、この値になるで、配列の最後の値がこれに一致する。
pr['fpol']

array([0.51131611, 0.50974103, 0.50816595, 0.50659087, 0.50501579,
       0.50344071, 0.50186563, 0.50029055, 0.49871547, 0.49714039,
       0.49556531, 0.49399023, 0.49241515, 0.49084007, 0.48926499,
       0.48768991, 0.48611483, 0.48453975, 0.48296467, 0.48138959,
       0.47981451, 0.47823944, 0.47666436, 0.47508928, 0.4735142 ,
       0.47193912, 0.47036404, 0.46878896, 0.46721388, 0.4656388 ,
       0.46406372, 0.46248864, 0.46091356, 0.45933848, 0.4577634 ,
       0.45618832, 0.45461324, 0.45303816, 0.45146308, 0.449888  ,
       0.44831292, 0.44673784, 0.44516276, 0.44358768, 0.4420126 ,
       0.44043752, 0.43886244, 0.43728736, 0.43571228, 0.4341372 ,
       0.43256212, 0.43098704, 0.42941196, 0.42783688, 0.4262618 ,
       0.42468672, 0.42311164, 0.42153656, 0.41996148, 0.4183864 ,
       0.41681132, 0.41523624, 0.41366116, 0.41208608, 0.410511  ,
       0.40893592, 0.40736084, 0.40578576, 0.40421068, 0.4026356 ,
       0.40106052])

#### ポロイダルカレントの式

$$
B=\frac{\mu_{0}}{2 \pi R} I
$$
から
$$
I=\frac{2 \pi}{\mu_{0}} R B
$$
g_fileの説明によれば
$$
F=R B_{T}=\frac{\mu_{0}}{2 \pi}I
$$